In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
import boto3
import sys
import re
from datetime import datetime
import configparser

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1669873530948_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
spark.sql('''
                    SELECT Distinct
                    (bigint(to_timestamp(ChocksOn)) - bigint(to_timestamp(ChocksOff))) / 60 AS minutes
                    From precogdb.q400128wps
                    WHERE filename = 'ALL ac=VT-SUQ dep=GOI arr=BLR fn=SG 2910 dt=20221120.csv'


''').show()

#spark.sql("""select unix_timestamp(DATE'2021-10-13')-unix_timestamp(DATE'2021-10-01') as seconds""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|minutes|
+-------+
|   90.0|
+-------+

In [17]:
spark.sql('''

            SELECT Distinct
            to_timestamp(ChocksOff) AS Actual_DEP_Time,
            to_timestamp(ChocksOn) AS Actual_Arr_Time
            From precogdb.q400128wps
            WHERE filename = 'ALL ac=VT-SUQ dep=GOI arr=BLR fn=SG 2910 dt=20221120.csv'
            

''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+
|    Actual_DEP_Time|    Actual_Arr_Time|
+-------------------+-------------------+
|2022-11-20 22:05:00|2022-11-20 23:35:00|
+-------------------+-------------------+

In [23]:
spark.sql('''

            SELECT Distinct
            Cast(ChocksOff as Date) AS FlightDate,
                                                ChocksOff,
                                                Departure,
                                                date_format(ChocksOff, 'HH:mm:ss') AS Actual_DEP_Time ,
                                                Arrival,
                                                date_format(ChocksOn, 'HH:mm:ss') AS Actual_ARR_Time 
            From precogdb.q400128wps
            WHERE filename = 'ALL ac=VT-SUQ dep=GOI arr=BLR fn=SG 2910 dt=20221120.csv'
            

''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+---------+---------------+-------+---------------+
|FlightDate|          ChocksOff|Departure|Actual_DEP_Time|Arrival|Actual_ARR_Time|
+----------+-------------------+---------+---------------+-------+---------------+
|2022-11-20|2022-11-20 22:05:00|      GOI|       22:05:00|    BLR|       23:35:00|
+----------+-------------------+---------+---------------+-------+---------------+

In [2]:
Q400Interim = spark.sql('''
                    SELECT * 
                        From precogdb.q400512wps where filename = 'ALL ac=VT-SUV dep=GWL arr=BLR fn=SG 3326 dt=20220407.csv' 
                ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
Q400Interim.createOrReplaceTempView("Q400Interim")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
FirstQuery  = "(SELECT FileName, max(cast(`RAD-ALT` as int)) as RADALT from Q400Interim group by FileName);"

spark.sql(FirstQuery).createOrReplaceTempView("B")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
SecondQuery = "(SELECT a.*, B.RADALT from Q400Interim a inner join B on a.FileName = B.FileName and a.`RAD-ALT` = B.RADALT)"

spark.sql(SecondQuery).createOrReplaceTempView("C")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
ThirdQuery = "(SELECT FileName, RADALT, max(`Time-In-Secs`) as MaxTime, min(`Time-In-Secs`) as MinTime from C group by FileName, RADALT)"

spark.sql(ThirdQuery).createOrReplaceTempView("D")

DF1 = spark.sql('''SELECT * FROM D''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
DF1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+-------+
|            FileName|RADALT|MaxTime|MinTime|
+--------------------+------+-------+-------+
|ALL ac=VT-SUV dep...|  2491| 9748.0| 9748.0|
+--------------------+------+-------+-------+

In [8]:
FourthQuery = '''(SELECT a.`Time-In-Secs` as `Time-In-Secs-EX`,
                        a.`BARO-CORRECTED-PALT-DERIVED` as `BARO-CORRECTED-PALT-DERIVED-EX`,
                    a.`RAD-ALT` as `RAD-ALT-EX`,
                    REPLACE(a.`ILS-GS-DEV-2`,'*','') as `ILS-GS-DEV-2-EX`,
                    REPLACE(a.`ILS-GS-DEV-1`,'*','') as `ILS-GS-DEV-1-EX`,
                    'G/SDeviation' as ExceedanceName,
                    CAST(CAST(unix_timestamp(CAST(a.ChocksOff AS Timestamp)) AS INT)+CAST(a.`Time-In-Secs` AS INT) AS TimeStamp) AS OccurenceTime,
                    case 
                    when Cast(`Time-In-Secs` as int)<D.MinTime Then 'TakeOff'
                    when Cast(`Time-In-Secs` as int) between D.MinTime and D.MaxTime Then 'Airborne'
                    when Cast(`Time-In-Secs` as int)>D.MaxTime Then 'Approach' end Phase,
                     
                    case when
                    case when replace(`ILS-GS-DEV-2`,'*','') like 'FU%' then 
                    CASE WHEN instr(replace(`ILS-GS-DEV-2`,'*',''),' ')>0 
                             THEN SUBSTRING(replace(`ILS-GS-DEV-2`,'*',''),
                             instr(replace(`ILS-GS-DEV-2`,'*',''),' ')+1,
                             LENGTH(replace(`ILS-GS-DEV-2`,'*','')))  
                             ELSE NULL END end > 1.00 and 
                    Cast(`Time-In-Secs` as int)>D.MaxTime Then 'High' else 'Low' end TriggerCondition1,
                    
                    case when
                    case when replace(`ILS-GS-DEV-1`,'*','') like 'FU%' then 
                    CASE WHEN instr(replace(`ILS-GS-DEV-1`,'*',''),' ')>0 
                             THEN SUBSTRING(replace(`ILS-GS-DEV-1`,'*',''),
                             instr(replace(`ILS-GS-DEV-1`,'*',''),' ')+1,
                             LENGTH(replace(`ILS-GS-DEV-1`,'*',''))) 
                             ELSE NULL END end > 1.00 and 
                    Cast(`Time-In-Secs` as int)>D.MaxTime Then 'High' else 'Low' end TriggerCondition2,
        
                    a.* from Q400Interim a
                    inner join D  
                    on a.FileName = D.FileName
                    where Cast(`Time-In-Secs` as int)>D.MaxTime and`WOW-MLG` = 'Air (0)')'''

spark.sql(FourthQuery).createOrReplaceTempView("E")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
FifthQuery = f'''(select * FROM E WHERE TriggerCondition1 = 'High' OR TriggerCondition2 = 'High' and `RAD-ALT` < 1000 and Phase = 'Approach' )'''

spark.sql(FifthQuery).createOrReplaceTempView("F")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
Q400SQLDF4 = spark.sql('''SELECT * FROM F''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
Q400SQLDF4.show()#count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------------------+----------+---------------+---------------+--------------+-------------------+--------+-----------------+-----------------+--------+-----------------+----------------+----------+--------+------+---------+------+----------------+-----+-------------------+-------------------+------+------------+-------------+-------+------+---------+-------+--------+----------+------------+------------+---------------------------+--------------+----------------+-----------+----------+-------+-------+-------+-----------+------------+-----------------+--------------+-----------------+----------------------+---------------+----------------------+-------------+-----+-----+--------+--------+-----+-----+----+----+-------------------+--------------+------------+-------------+--------------+------------+------------+--------------+--------------+----------+----------+--------------+--------------+----------------+-----------------+-----------------+----------------